In [ ]:
import tensorflow as tf
import numpy as np
from mlp.data_providers import AugmentedCIFAR10DataProvider, AugmentedCIFAR100DataProvider, CIFAR100DataProvider, CIFAR10DataProvider
from mlp.image_transforms import random_flip, random_crop, center_crop, random_flip_small
from mlp.Conv_models import ConvModel, TwoTaskConvModel
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

seed = 666 
rng = np.random.RandomState(seed)

In [ ]:
model_name = 'mtl-classifier-fast-long'

In [ ]:
def combined_transformer(inputs, rng):
    inputs = random_crop(inputs, rng)
    return random_flip_small(inputs, rng)

In [ ]:
layer_dims = [3, 64, 128, 256, 512, _]
batch_size = 256

main_train_data = CIFAR100DataProvider(which_set='train',
                                                batch_size=batch_size, shuffle_order=False)
main_valid_data = CIFAR100DataProvider(which_set='valid', batch_size=batch_size, shuffle_order=False)

aux_train_data = CIFAR100DataProvider(which_set='train',
                                               batch_size=batch_size, shuffle_order=False, use_coarse_targets=True)
aux_valid_data = CIFAR100DataProvider(which_set='valid', batch_size=batch_size, 
                                      use_coarse_targets=True, shuffle_order=False)

bifurcation_point = 2
joint_loss = False
max_pools = [True, True, False, True]
lrns = []
lrn_alphas = []
batch_norms = []
l1 = False
l2 = 5e-4
learning_rates = [2e-4]  
lrn_schedule = []
optimizer_params = [0.9, 0.999]  
activation = tf.nn.relu
dropout = True
epochs = 800
error = 'soft_max_cross_entropy'
image_size = 32  
optimiser = "Adam"  
logs_dir = "tf-log"

In [ ]:
model = TwoTaskConvModel(conv_layer_dims=layer_dims, main_train_data=main_train_data, main_valid_data=main_valid_data,
                         aux_train_data=aux_train_data, aux_valid_data=aux_valid_data, L1=l1, L2=l2,
                         batch_size=batch_size, learning_rates=learning_rates, learning_rate_schedule=lrn_schedule,
                         optimiser=optimiser, max_pools=max_pools, lin_response_norms=lrns,
                         lin_response_alphas=lrn_alphas, batch_norms=batch_norms, bifurcation_point=bifurcation_point,
                         optimiser_params=optimizer_params, activation=activation, dropout=dropout,
                         epochs=epochs, error=error, input_image_size=image_size, name=model_name, joint_loss=joint_loss)
model.create_network()
model.initialize_network(logs_dir)
model.train_model()

In [ ]:
tf.reset_default_graph()

In [ ]:
model_name = 'baseline-fast-long'

In [ ]:
train_data = CIFAR100DataProvider(which_set='train', batch_size=batch_size)
valid_data = CIFAR100DataProvider(which_set='valid', batch_size=batch_size)
epochs = 800

In [ ]:
model = ConvModel(conv_layer_dims=layer_dims, train_data=train_data, valid_data=valid_data,
                 batch_size=batch_size, learning_rates=learning_rates, learning_rate_schedule=[],
                 optimiser=optimiser, L1=l1, L2=l2,
                 max_pools=max_pools, lin_response_norms=lrns, lin_response_alphas=lrn_alphas, batch_norms=batch_norms,
                 optimiser_params=optimizer_params, activation=activation, dropout=dropout,
                 epochs=epochs, error=error, input_image_size=image_size, name=model_name)
model.create_network()
model.initialize_network(logs_dir)
model.train_model()